# Single Run

In [1]:
import pandas as pd

from surgeryschedulingunderuncertainty.task import Task
from surgeryschedulingunderuncertainty.patients_provider import PatientsFromHistoricalDataProvider
from surgeryschedulingunderuncertainty.master import Master

from surgeryschedulingunderuncertainty.implementor import StandardImplementor
from surgeryschedulingunderuncertainty.optimizer import ImplementorAdversary

from surgeryschedulingunderuncertainty.predictive_model import NGBLogNormal, NGBNormal


/var/folders/w_/kqymfxcs5d561j515w6_cfqh0000gn/T/ipykernel_42007/317628422.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Task: pazienti storici

In [2]:
historical_data_df = pd.read_csv("../data/historical_data.csv")

patient_provider = PatientsFromHistoricalDataProvider(
                 historical_data= historical_data_df
                 )

In [3]:
or_master_schedule = pd.read_csv("../data/master_schedule_input.csv", sep = ';')
master_schedule  = Master(name = "my master", table=or_master_schedule)

In [4]:
num_of_patients = 100

In [5]:
patients, training = patient_provider.provide_sets(quantity=num_of_patients, quantity_training=1000)


In [6]:
model = NGBLogNormal(description="test", patients=training)
patients = model.predict(patients)

In [7]:

task = Task(name = "My first problem",
            num_of_weeks= 2,
            num_of_patients= num_of_patients,
            robustness_risk= 0.2,
            robustness_overtime= 10,
            urgency_to_max_waiting_days= {0: 7, 1:30, 2:60, 3:180, 4:360}, 
            )

task.patients = patients
task.master_schedule = master_schedule


### Implementor - Adversary

In [8]:
implementor_adversary = ImplementorAdversary(task = task, implementor=StandardImplementor(), adversary=None, description = 'Test')

In [9]:
implementor_adversary.create_instance()

In [10]:
solution_implementor_adversary = implementor_adversary.run(15)

implementor
adversary


In [11]:
for pat in (solution_implementor_adversary._blocks[0].patients):
    print(pat)

Patient id: 3425 
 equipe: D 
 urgency: 4 
 nominal duration: 83 
 days waiting: 115


solution._blocks[4].patients[0].uncertainty_profile.param_s

### Chance constraint

In [12]:
from surgeryschedulingunderuncertainty.optimizer import VanillaImplementor
from surgeryschedulingunderuncertainty.implementor import ChanceConstraintsImplementor


In [13]:
chance_constraints = VanillaImplementor(task = task, 
                                              implementor=ChanceConstraintsImplementor(task = task), 
                                              description = 'Test')

In [14]:
chance_constraints.create_instance()

In [15]:
solution_chance_constraints = chance_constraints.run()

In [16]:
for pat in (solution_implementor_adversary._blocks[0].patients):
    print(pat)

Patient id: 3425 
 equipe: D 
 urgency: 4 
 nominal duration: 83 
 days waiting: 115


In [17]:
from surgeryschedulingunderuncertainty.optimizer import  BudgetSet
from surgeryschedulingunderuncertainty.implementor import  BSImplementor


In [18]:
budget_set = BudgetSet(task = task, 
                              implementor=BSImplementor(task = task), 
                              description = 'Test')






In [19]:
budget_set.run()

implementor


In [20]:
budget_set.create_instance()

In [21]:
solution_budget_set = budget_set.run_implementor()

In [22]:
for pat in (solution_budget_set._blocks[2].patients):
    print(pat.uncertainty_profile.nominal_value)
    print(pat.uncertainty_profile.param_s)

193.69660099910917
0.32612209944949433
177.0974650464461
0.3312980215971208
181.80914834589376
0.31733940253247667
